In [1]:
using Optim
using Random
using StatsBase
using Statistics
using Distributions

In [2]:
function creat_players(clubs, players_per_club, mean = 0, var = 1, lb = 0, ub = 100)
    all_players = rand(Truncated(Normal(mean, var), lb, ub), clubs * players_per_club)
    all_players = all_players / (sum(all_players) / clubs)
    players = Dict{Int64, Float64}(zip(1:clubs * players_per_club, all_players))
    return players
end

function simulating_game(club1, club2, sims = 1000000)
    λ₁ = sum(club1)
    λ₂ = sum(club2)
    X₁ = Poisson(λ₁ / λ₂)
    X₂ = Poisson(λ₂ / λ₁)
    Y₁ = rand(X₁, sims)
    Y₂ = rand(X₂, sims)
    wins₁ = sum(Y₁ .> Y₂)
    draws = sum(Y₁ .== Y₂)
    wins₂ = sum(Y₁ .< Y₂)
    return wins₁, draws, wins₂
end

function create_games(seasons, clubs, ppc)
    results = [[[0 for i in 1:4] for j in 1:clubs * (clubs - 1)] for s in 1:seasons]
    players = creat_players(clubs, ppc)
    squads = []
    for i in 1:seasons
        line = 1
        append!(squads, [reshape(shuffle(collect(1:length(players))), (clubs, ppc))])
        actual_clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
        for i in 1:clubs
            for j in 1:ppc
                actual_clubs[i, j] = players[actual_clubs[i, j]]
            end
        end

        for j in 1:clubs
            for k in 1:clubs
                if j != k
                    Xⱼ = Poisson(sum(actual_clubs[j, :]) / sum(actual_clubs[k, :]))
                    Xₖ = Poisson(sum(actual_clubs[k, :]) / sum(actual_clubs[j, :]))
                    results[i][line][1] = j
                    results[i][line][2] = rand(Xⱼ)
                    results[i][line][3] = rand(Xₖ)
                    results[i][line][4] = k
                    line += 1
                end
            end
        end
    end
    return results, squads, players
end

# results be an array like [1, 1, 0, 3]
# [home club, score home, score away, away club]

function likelihood(results, squads, players)
    if typeof(players) != Dict{Int64, Float64}
        players = Dict{Int64, Float64}(zip(1:length(players), players))
    end
    seasons = length(results)
    loglikelihood = 0
    for i in 1:seasons
        games = length(results[i])
        actual_clubs = convert(Matrix{Float64}, (deepcopy(squads[i])))
        for j in 1:size(actual_clubs)[1]
            for k in 1:size(actual_clubs)[2]
                actual_clubs[j, k] = players[actual_clubs[j, k]]
            end
        end
        
        for j in 1:games
            clubₕ, scoreₕ, scoreₐ, clubₐ = results[i][j]
            loglikelihood -= logpdf(Poisson(sum(actual_clubs[clubₕ, :]) / sum(actual_clubs[clubₐ, :])),
                                    scoreₕ)
            loglikelihood -= logpdf(Poisson(sum(actual_clubs[clubₐ, :]) / sum(actual_clubs[clubₕ, :])),
                                    scoreₐ)
        end
    end
    
    return loglikelihood
end

likelihood (generic function with 1 method)

In [3]:
seasons = 10
n_clubs = 20
ppc = 11

# compilando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(results, squads, players)

# reexecutando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(results, squads, players)

  1.265584 seconds (2.14 M allocations: 126.719 MiB, 8.02% gc time, 99.09% compilation time)
  0.528276 seconds (2.55 M allocations: 146.268 MiB, 9.32% gc time, 98.16% compilation time)
  0.011129 seconds (69.11 k allocations: 3.847 MiB)
  0.010904 seconds (83.26 k allocations: 3.674 MiB)


9967.608772834299

# Teste para Otimização

In [4]:
goal(x, results, squads) = likelihood(results, squads, x)
lower = zeros(220)
upper = 20 * ones(220)
x_inicial = rand(220)
# r = optimize(x -> goal(x, results, squads), x_inicial, lower, upper, Fminbox{NelderMead}())
# res = optimize(x -> goal(x, results, squads), x_inicial, Optim.Options(iterations = 10000))
res = optimize(x -> goal(x, results, squads),
               lower,
               upper,
               x_inicial,
               Fminbox(NelderMead()),
               Optim.Options(iterations = 1000))

res

 * Status: success

 * Candidate solution
    Final objective value:     9.889672e+03

 * Found with
    Algorithm:     Fminbox with Nelder-Mead

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.99e+01 ≰ 1.0e-08

 * Work counters
    Seconds run:   1804  (vs limit Inf)
    Iterations:    106
    f(x) calls:    180149
    ∇f(x) calls:   1


In [5]:
Optim.converged(res), Optim.minimizer(res), Optim.minimum(res)

(true, [1.1133180627705355, 0.9945806855203053, 0.6412189922492487, 0.23572001485143834, 1.4018098519618056, 0.8661142055945934, 0.7156322589597405, 1.3301574724407956, 0.13161616989968333, 0.7061975907423915  …  0.4740729728176876, 0.32527846527492077, 1.3622133049057856, 0.6661103937617975, 0.6658818407191314, 0.19356471110733547, 1.7941310611438703, 0.6285986250596511, 1.648434883861554, 1.306276732878969], 9889.671503047166)

In [6]:
max_lik_players = Dict{Int64, Float64}(zip(1:length(Optim.minimizer(res)), Optim.minimizer(res)))

original_players = zeros(220)
optimized_players = zeros(220)

for i in 1:220
    original_players[i] = players[i]
    optimized_players[i] = max_lik_players[i]
end

original_players /= original_players[1]
optimized_players /= optimized_players[1]

println("Correlação de Pearson: ", cor(original_players, optimized_players))
println("Correlação de Spearman: ", corspearman(original_players, optimized_players))
println("Correlação de Kendall: ", corkendall(original_players, optimized_players))

Correlação de Pearson: 0.7619505713757363
Correlação de Spearman: 0.7139662915650022
Correlação de Kendall: 0.5186384391863844
